In [9]:
from __future__ import division

import numpy as np 
import tensorflow as tf
import keras.backend as K
from keras.models import Model
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam, SGD
from deform_conv.layers import ConvOffset2D
from deform_conv.callbacks import TensorBoard
from deform_conv.cnn import get_cnn, get_deform_cnn
from deform_conv.mnist import get_gen

from matplotlib import pyplot as plt
import os

In [2]:
# Tensorflow config 
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.Session(config=config)
K.set_session(sess)

In [3]:
# ---
# Config
batch_size = 32
n_train = 60000
n_test = 10000
steps_per_epoch = int(np.ceil(n_train / batch_size))
validation_steps = int(np.ceil(n_test/ batch_size))

In [4]:
train_gen = get_gen(
    'train', batch_size=batch_size,
    scale=(1.0, 1.0), translate=0.0,
    shuffle=True
)

test_gen = get_gen(
    'test', batch_size=batch_size,
    scale=(1.0, 1.0), translate=0.0,
    shuffle=False
)

train_scaled_gen = get_gen(
    'train', batch_size=batch_size,
    scale=(1.0, 2.5), translate=0.2,
    shuffle=True
)

test_scaled_gen = get_gen(
    'test', batch_size=batch_size,
    scale=(1.0, 2.5), translate=0.2,
    shuffle=False
)

# Normal CNN

In [5]:
inputs, outputs = get_cnn()
model = Model(inputs=inputs, outputs=outputs)

Instructions for updating:
Colocations handled automatically by placer.


In [6]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv11_relu (Activation)     (None, 28, 28, 32)        0         
_________________________________________________________________
conv11_bn (BatchNormalizatio (None, 28, 28, 32)        128       
_________________________________________________________________
conv12 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
conv12_relu (Activation)     (None, 14, 14, 64)        0         
_________________________________________________________________
conv12_bn (BatchNormalizatio (None, 14, 14, 64)        256       
__________

In [7]:
optim = Adam(1e-3)
loss = categorical_crossentropy
model.compile(optim, loss, metrics=['accuracy'])

In [8]:
model.fit_generator(
    train_gen, steps_per_epoch=steps_per_epoch,
    epochs=10, verbose=1,
    validation_data=test_gen, validation_steps=validation_steps
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.1557 - acc: 0.9598 - val_loss: 0.0791 - val_acc: 0.9746
Epoch 2/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0489 - acc: 0.9860 - val_loss: 0.0447 - val_acc: 0.9853
Epoch 3/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0355 - acc: 0.9893 - val_loss: 0.0355 - val_acc: 0.9895
Epoch 4/10
1875/1875 [==============================] - 17s 9ms/step - loss: 0.0264 - acc: 0.9916 - val_loss: 0.0372 - val_acc: 0.9872
Epoch 5/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0207 - acc: 0.9933 - val_loss: 0.3080 - val_acc: 0.9077
Epoch 6/10
1875/1875 [==============================] - 18s 10ms/step - loss: 0.0170 - acc: 0.9947 - val_loss: 0.0369 - val_acc: 0.9877
Epoch 7/10
1875/1875 [==============================] - 19s 10ms/step - loss: 0.0139 - acc: 0.9954 - val_loss: 0.0296 - val_acc: 0.9896
Epo

In [11]:
if not os.path.exists('weights'):
    os.makedirs('weights')
    
model.save_weights('weights/cnn.h5')

In [12]:
# Evaluate normal CNN
val_loss, val_acc = model.evaluate_generator(
    test_gen, steps=validation_steps
)

print('Test accuracy', val_acc)

Test accuracy 0.9908


In [13]:
val_loss, val_acc = model.evaluate_generator(
    test_scaled_gen, steps=validation_steps
)
print('Test accuracy with scaled images', val_acc)

Test accuracy with scaled images 0.6449


# Deformable CNN

In [19]:
inputs, outputs = get_deform_cnn(trainable=False)
deform_model = Model(inputs=inputs, outputs=outputs)
deform_model.load_weights('weights/cnn.h5', by_name=True)

In [20]:
deform_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           (None, 28, 28, 1)         0         
_________________________________________________________________
conv11 (Conv2D)              (None, 28, 28, 32)        320       
_________________________________________________________________
conv11_relu (Activation)     (None, 28, 28, 32)        0         
_________________________________________________________________
conv11_bn (BatchNormalizatio (None, 28, 28, 32)        128       
_________________________________________________________________
conv12_offset (ConvOffset2D) (None, 28, 28, 32)        18432     
_________________________________________________________________
conv12 (Conv2D)              (None, 14, 14, 64)        18496     
_________________________________________________________________
conv12_relu (Activation)     (None, 14, 14, 64)        0         
__________

In [21]:
optim = Adam(5e-4)
loss = categorical_crossentropy
deform_model.compile(optim, loss, metrics=['accuracy'])

In [22]:
deform_model.fit_generator(
    train_scaled_gen, steps_per_epoch=steps_per_epoch,
    epochs=20, verbose=1,
    validation_data=test_scaled_gen, validation_steps=validation_steps
)

Epoch 1/20
1875/1875 [==============================] - 224s 120ms/step - loss: 0.5887 - acc: 0.8250 - val_loss: 0.3815 - val_acc: 0.8765
Epoch 2/20
1875/1875 [==============================] - 228s 121ms/step - loss: 0.3954 - acc: 0.8787 - val_loss: 0.3334 - val_acc: 0.8961
Epoch 3/20
1875/1875 [==============================] - 236s 126ms/step - loss: 0.3407 - acc: 0.8942 - val_loss: 0.2537 - val_acc: 0.9231
Epoch 4/20
1875/1875 [==============================] - 241s 128ms/step - loss: 0.3084 - acc: 0.9058 - val_loss: 0.2394 - val_acc: 0.9257
Epoch 5/20
1875/1875 [==============================] - 213s 114ms/step - loss: 0.2829 - acc: 0.9155 - val_loss: 0.2276 - val_acc: 0.9303
Epoch 6/20
1875/1875 [==============================] - 230s 122ms/step - loss: 0.2658 - acc: 0.9182 - val_loss: 0.2075 - val_acc: 0.9362
Epoch 7/20
1875/1875 [==============================] - 222s 118ms/step - loss: 0.2548 - acc: 0.9220 - val_loss: 0.1917 - val_acc: 0.9410
Epoch 8/20
1875/1875 [============

In [24]:
deform_model.save_weights('weights/deform_cnn.h5')

In [26]:
val_loss, val_acc = deform_model.evaluate_generator(
    test_scaled_gen, steps=validation_steps
)
print('Test accuracy of deformable convolution with scaled images', val_acc)

val_loss, val_acc = deform_model.evaluate_generator(
    test_gen, steps=validation_steps
)
print('Test accuracy of deformable convolution with regular images', val_acc)

Test accuracy of deformable convolution with scaled images 0.9425
Test accuracy of deformable convolution with regular images 0.9759
